In [1]:
from interp_utils import *

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as dists


In [121]:
from interp_utils import hist

alpha = 3
beta = 10

distr= dists.Beta(alpha, beta)


sample = distr.sample((10000,))

hist(sample)

In [460]:
reload_module('feature_tree')
from feature_tree import Tree


tree = Tree(n_growths=2, root_is_feature=True)

In [3]:
# import numpy as np


# class Node:
#     def __init__(self, is_read_out, is_allopatry):
#         super().__init__()
#         # random name
#         self.name = ''.join(np.random.choice(list('abcdefghijklmnopqrstuvwxyz'), 5))
#         self.is_read_out = is_read_out
#         self.is_allopatry = is_allopatry
#         self.edges = []
#     def __repr__(self):
#         return f'''Node (
#     is_read_out: {self.is_read_out}
#     is_allopatry: {self.is_allopatry}
#     edges: {self.pretty_child_probs}
# )'''
#     @property
#     def child_probs(self):
#         return torch.tensor([edge['child_prob'] for edge in self.edges])
#     @property
#     def pretty_child_probs(self):
#         return ', '.join([f'{edge["child_prob"]:.2f}' for edge in self.edges])
#     def __hash__(self):
#         return hash(self.name)
#     def create_child(self):
#         child_is_allopatry = bool(dists.Bernoulli(0.25).sample())
#         child_prob_dist = dists.Beta(9, 10) if child_is_allopatry else dists.Beta(5, 10)
#         if self.is_allopatry is False:
#             # sample from beta distribution
#             child_is_dense = bool(dists.Bernoulli(0.5).sample()) if child_is_allopatry else False
#             child_prob = 1.0 if child_is_dense else child_prob_dist.sample().item()
#             child_is_read_out = False if child_is_dense else True
#         else:
#             child_is_read_out = True
#             new_child_probs = torch.cat((self.child_probs, child_prob_dist.sample((1,))))
#             new_child_probs = new_child_probs/new_child_probs.sum()
#             assert torch.allclose(new_child_probs.sum(), torch.tensor(1.0))
#             # print(f'{len(self.edges)=}')
#             # print(f'{len(new_child_probs)=}')
#             for i, new_child_prob in enumerate(new_child_probs[:-1]):
#                 self.edges[i]['child_prob'] = new_child_prob
#             child_prob = new_child_probs[-1].item()

#         child_node = Node(is_read_out=child_is_read_out, is_allopatry=child_is_allopatry)

#         if child_is_allopatry:
#             child_node.create_child()
#             child_node.create_child()

#         self.edges.append({'child_prob': child_prob, 'child': child_node})

#         return self.edges[-1]['child']

#     def sample_child(self):
#         probs = self.child_probs
#         allopatry_values = [edge['child'].is_allopatry for edge in self.edges]
#         probs = (probs*(1+torch.tensor(allopatry_values, dtype=torch.float))).clamp(0, 1)
#         return self.edges[dists.Categorical(probs).sample()]['child']
#     def add_feature(self):
#         locally_add_feature = bool(dists.Bernoulli(0.65).sample()) if len(self.edges) > 0 else True
#         if locally_add_feature:
#             self.create_child()
#         else:
#             self.sample_child().add_feature()    
    

# node = Node(True, True)

# for _ in range(30):
#     node.add_feature()


# node.child_probs



tensor([0.0042, 0.0018, 0.0024, 0.0026, 0.0026, 0.0051, 0.0040, 0.0062, 0.0146,
        0.0184, 0.0095, 0.0249, 0.0453, 0.0706, 0.0514, 0.1262, 0.1333, 0.1983,
        0.2786])

In [5]:
# from treelib import Node, Tree

# tree = Tree()

# tree.create_node('root', 'root')

# def recursively_add_nodes(node, parent_name):
#     for edge in node.edges:
#         is_read_out, is_allopatry = edge['child'].is_read_out, edge['child'].is_allopatry
#         child_name = ''
#         if not is_read_out:
#             child_name += '_'
#         if is_allopatry:
#             child_name += 'A'
#         child_name += f" {round(float(edge['child_prob']), 4)}"
#         tree.create_node(child_name, edge['child'].name, parent=parent_name)
#         recursively_add_nodes(edge['child'], edge['child'].name)

# recursively_add_nodes(node, 'root')
# tree.show()

root
├──  0.0026
├──  0.0026
├──  0.004
│   └──  0.5038
├──  0.0042
│   ├──  0.4019
│   │   └──  0.181
│   └── A 0.5584
│       ├──  0.5861
│       └── A 0.4139
│           ├──  0.2377
│           └──  0.7623
├──  0.0051
├──  0.0062
├──  0.0095
├──  0.0146
├──  0.0249
├──  0.0453
│   └──  0.631
├──  0.0514
├──  0.0706
├──  0.1983
│   └── A 0.2653
│       ├──  0.349
│       └──  0.651
├──  0.2786
├── A 0.0018
│   ├──  0.1658
│   └──  0.8342
│       └──  0.0974
├── A 0.0024
│   ├──  0.2645
│   └──  0.7355
│       └── A 0.4565
│           ├──  0.1672
│           └──  0.8328
├── A 0.0184
│   ├──  0.3852
│   └── A 0.6148
│       ├──  0.2493
│       └── A 0.7507
│           ├──  0.2479
│           └── A 0.7521
│               ├──  0.2585
│               └──  0.7415
├── A 0.1262
│   ├──  0.0793
│   ├──  0.2002
│   ├──  0.4759
│   └── A 0.2446
│       ├──  0.1928
│       ├──  0.289
│       └── A 0.5182
│           ├──  0.3014
│           └── A 0.6986
│               ├──  0.6626
│              

root
├── 0.02
├── 0.02
├── 0.04
├── 0.06
├── 0.07
├── 0.1
├── 0.1
├── 0.17
├── 0.21
└── 0.21
    └── 0.22



In [54]:
node.edges

[{'child_prob': tensor(0.0807),
  'child': Node (
      is_read_out: True
      is_allopatry: True
      edges: 1.00, 1.00
  )},
 {'child_prob': tensor(0.0303),
  'child': Node (
      is_read_out: True
      is_allopatry: True
      edges: 0.65, 0.35
  )},
 {'child_prob': tensor(0.0379),
  'child': Node (
      is_read_out: True
      is_allopatry: True
      edges: 0.52, 0.29
  )},
 {'child_prob': tensor(0.0360),
  'child': Node (
      is_read_out: True
      is_allopatry: False
      edges: 
  )},
 {'child_prob': tensor(0.0565),
  'child': Node (
      is_read_out: True
      is_allopatry: False
      edges: 
  )},
 {'child_prob': tensor(0.0626),
  'child': Node (
      is_read_out: True
      is_allopatry: False
      edges: 
  )},
 {'child_prob': tensor(0.0687),
  'child': Node (
      is_read_out: True
      is_allopatry: False
      edges: 
  )},
 {'child_prob': tensor(0.1390),
  'child': Node (
      is_read_out: True
      is_allopatry: True
      edges: 0.16, 0.26
  )},
 {'c